In [1]:
import numpy as np
import sklearn

In [3]:
def get_actions(action_clf, data, desired_probability, epochs=10, alpha=0.1, epsilon=0.0001,
                first_k = 100, labels=None, verbose=False): 
    #In the future, include a feature_mask instead of first_k
    #This returns all actions. May want to create something that does top-n actions.
    """
    action_clf: a classifier that is compatible with sklearn, typically the overall classifier
    data: a single row of bill data (numpy array)
    desired_probability: an float between 0 and 1 inclusive
    first_k: only act on (and print) the first k actions
    labels: the named labels of the inputs
    base_labels: stemmed labels (i.e. not split into pd dummies)
    epochs: number of epochs to run iterative gradient descent
    alpha: step size to move in the direction of the gradient
    epsilon: the size of the perturbation of the inputs
    verbose: if True, will print predicted probability at each epoch, and print final actions
    """
    
    initial_prob = action_clf.predict_proba(data)
    
    if verbose:
        print("Initial probability of passing:", initial_prob)
    
    DZDY = desired_probability - initial_prob #gradient of the loss wrt the output. Scalar for a single input
    DZDX = np.zeros_like(data) #empty array of the same size as data (i.e. x)
    DZDX_c = np.zeros_like(data)
    
    x_p = data.copy()
    x_n = data.copy()

    for epoch in range(epochs):
        #can handle batches of data (multiple bills):
        for t, row in enumerate(x_p): 
            #since our dataset is metadata followed by tfidf words, and we can't really affect the words yet:
            for i in range(first_k): 
                #perturb the inputs a little bit
                x_p[t][i] += epsilon
                x_n[t][i] -= epsilon
                #predict from the perturbed input
                f_p = action_clf.predict_proba(x_p)
                f_n = action_clf.predict_proba(x_n)
                #this is basically newtons method to find the gradient
                DZDX[t][i] = np.sum(DZDY*((f_p-f_n)/(2*epsilon)))
                # reset the tensors to their original state 
                x_p[t][i] -= epsilon
                x_n[t][i] += epsilon
                
        DZDX_c += DZDX
        adjusted_data = data + DZDX_c
        adjusted_data[adjusted_data < 0] = 0
        DZDX_c = adjusted_data - data
        
        new_prob = action_clf.predict_proba(data + DZDX_c)
        
        if verbose:
            print("Prob on epoch", epoch, ":", new_prob)
            
        DZDY = desired_probability - new_prob
    
    if verbose:
        print("Final actions:")
        for i in range(first_k):
            print("{0:.5f}".format(DZDX_c[0][i]), "\t", labels[i])

    return DZDX_c
    
    
def get_recommendations(clf, data, desired_probability, DZDX_c, label_action_count, 
                        metadata_num, labels, base_labels, verbose=False):
    """
    get_recommendations basically turns get_actions into useful information
    clf: sklearn classifier, typically the overall classifier, built for a MLP (neural network)
    data: 
    desired_probability: between 0 and 1
    DZDX_c: the cumulative change in the variable vector as produced by get_actions
    metadata_num: number of selected dummy metadata features (selects the first metadata_num features)
    base_labels: the base labels that pd.dummies has made expanded into one-hot encodings
    label_action_count: an iterable of ints indicating the number of options to give for that feature. 
        Has the same length as base_labels. 
        ex: base_labels = [committee, primary_sponsor], label_action_count = [3, 10]
    """
    feature_importances = {base_label:[] for base_label in base_labels}
    for i, base_label in enumerate(base_labels):
        #this works because pd dummies puts things in order
        for j in range(metadata_num):
            if labels[j][0:len(base_label)] == base_label:
                #store in feature_importances such that they can be sorted later
                feature_importances[base_label].append((labels[j], DZDX_c[0][j]))
            
    #sort each label by absolute feature importance
    for i, label in enumerate(feature_importances):
        #sort by the absolute value of that feature importance 
        feature_importances[label].sort(key=lambda x: abs(x[1]), reverse=True)
    
    final_actions = {base_label:[] for base_label in base_labels}
    for i, count in enumerate(label_action_count):
        final_actions[base_labels[i]] = feature_importances[base_labels[i]][0:count]
    
    if verbose:
        for key in final_actions.keys():
            
            if final_actions[key] != []:
                print(key)
                for i in final_actions[key]:
                    print("\t", "{0:.5f}".format(i[1]), "\t\t", i[0])

    return final_actions




#### FUTURE WORK BELOW ####
        
def what_if_from_recs(clf, data, modifications, top_n=10):
    """
    A function that takes a classifier, some data, and modifications to that data, and 
    Produces a cartesian product of modifications. i.e. if modifications is {'a':['b','c'],'d':['e','f','g']},
    then 6 probabilities will be produced
    
    send in modicications produced by get_recommendations
    """
    modification_calls = modifications.keys()
    
    for call in modification_calls:
        modifications[call].append('') #the 'no change' call

    return  



def what_if_from_recs_helper(clf, data, modifications, top_n=10, score_dict={}):
    """Create a list of probabilities from the list of recommendations."""
    pass
        
        

def individual_what_if_from_recs(clf, data, modifications, top_n=10):
    """
    A function that takes a classifier, some data, and modifications to that data, and 
    Produces a cartesian product of modifications. i.e. if modifications is {'a':['b','c'],'d':['e','f','g']},
    then 6 probabilities will be produced
    
    send in modicications produced by get_recommendations
    """
    modification_calls = modifications.keys()
    
    for call in modification_calls:
        for modification in modifications[call]:
            new_data = data[modification[0]]
            probability = clf.predict_proba()
            
           
    
    
    return      
        
def what_if(clf, data, modifications):
    """
    A function that takes a classifier, some data, and modifications to that data, and 
    Produces a cartesian product of modifications. i.e. if modifications is {'a':['b','c'],'d':['e','f','g']},
    then 6 probabilities will be produced
    """
    return
    

In [ ]:
#example usage
